In [49]:
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader

In [51]:
transform = T.Compose([T.ToTensor(), T.Normalize((0.5,),(0.5,))])
train_set = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
trainloader = DataLoader(train_set, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [75]:
x = trainloader._get_iterator().__next__()
x = x[0]
x = nn.Conv2d(x.shape[1], 64, 3, 1, 3)(x)
x = nn.LeakyReLU(0.1)(x)

x = nn.Conv2d(x.shape[1], x.shape[1]//2, 3, 2, 1)(x)
x = nn.LeakyReLU(0.1)(x)

x = nn.Conv2d(x.shape[1], x.shape[1]//2, 3, 1, 1)(x)
x = nn.LeakyReLU(0.1)(x)

x = nn.MaxPool2d((2,2),(2,2))(x)

x = torch.flatten(x,1)
x = nn.Linear(x.shape[1], x.shape[1]//2)(x)
x = nn.Linear(x.shape[1], x.shape[1]//2)(x)
x = nn.Linear(x.shape[1], 10)(x)


(2,)
torch.Size([64, 1, 28, 28])
torch.Size([64, 64, 32, 32])
torch.Size([64, 32, 16, 16])
torch.Size([64, 16, 16, 16])
torch.Size([64, 16, 8, 8])


torch.Size([64, 1024])

In [64]:
x.shape

torch.Size([64, 28, 7, 7])

In [84]:
16*7*7

784

In [88]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CnnNet(nn.Module):
    def __init__(self):
        super(CnnNet, self).__init__()
        # Define Convolutional layers
        self.conv1 = nn.Conv2d(1, 64, 3, 1, 3)  # Assuming input has 1 channel (e.g., MNIST)
        self.conv2 = nn.Conv2d(64, 32, 3, 2, 1) # Adjusted to fixed channels
        self.conv3 = nn.Conv2d(32, 16, 3, 1, 1) # Adjusted to fixed channels

        # Define Fully Connected layers
        # Note: You need to calculate the correct input size for the first linear layer
        self.fc1 = nn.Linear(1024, 512) # Example size, adjust as needed
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x): 
        # Conv 1
        x = F.leaky_relu(self.conv1(x), 0.1)

        # Conv 2
        x = F.leaky_relu(self.conv2(x), 0.1)

        # Conv 3
        x = F.leaky_relu(self.conv3(x), 0.1)

        # Max_pooling
        x = F.max_pool2d(x, (2, 2))

        # FC layers
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x), 0.1)
        x = F.leaky_relu(self.fc2(x), 0.1)
        x = F.leaky_relu(self.fc3(x), 0.1)
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net = CnnNet()
      

In [89]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [90]:
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')
print('Finished Training')

Epoch 1, Loss: 0.1682757355088094
Epoch 2, Loss: 0.051945367192102125
Epoch 3, Loss: 0.03591944213317055
Epoch 4, Loss: 0.029504554646627938
Epoch 5, Loss: 0.02420662896157179
Finished Training


In [93]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 99.04 %
